In [1]:
import argparse
import logging

import os
import numpy as np
import torch
import tqdm

In [2]:
from transformers import (
    GPT2TimeLMHeadModel,
    GPT2Tokenizer,
)

In [3]:
import sys
sys.path.append('../language-modeling')
from run_time_clm import (
    get_checkpoint,
    get_special_tokens,
    get_data_paths,
    get_dataset)

sys.path.append('../text-generation')
from generation_metrics import GenerationMetrics

2022-11-16 19:39:29.430893: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [5]:
MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

MODEL_CLASSES = {
    "gpt2": (GPT2TimeLMHeadModel, GPT2Tokenizer),
}

In [16]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)
        
def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length

def simulate_brownian_bridge(B_0, B_T, num_samples, sentence_lengths, dt=0.05, mu=0.0, sigma=1.0):
    """Run bridge forward pinned at B_0 and B_T"""
    if isinstance(B_0, torch.Tensor):
        B_0 = B_0.cpu().detach().numpy()
    if isinstance(B_T, torch.Tensor):
        B_T = B_T.cpu().detach().numpy()

    bridge = [B_0]
    x_t = np.copy(B_0)
    for step in range(num_samples - 2): # number of sentences
        dim = B_0.shape[-1]
        noise = np.sqrt(dt)*sigma*np.random.normal(mu, sigma, dim)
        t = step/num_samples
        x_tp1 = x_t * (1- dt/(1-t)) + (dt/(1-t))*B_T + noise
        length_idx = step % len(sentence_lengths)
        bridge += [x_tp1] * sentence_lengths[length_idx]
        x_t = x_tp1

    length_idx = step % len(sentence_lengths)
    bridge += [B_T] * sentence_lengths[length_idx]

    return bridge

def split_text(raw_text):
    split_pattern = ". "
    split_raw_text = [_ + split_pattern for _ in raw_text.split(split_pattern)]
    split_raw_text[-1] = split_raw_text[-1].rstrip(split_pattern)
    return split_raw_text

def get_density(dataset, lm, cl_model):
    """Estimate density of last latent"""
    first_latents = []
    last_latents = []
    length = len(dataset)
    print(length)
    for text_i in range(length):
        first_latents.append(dataset.cl_embeddings[text_i][0].detach().cpu().numpy())
        last_latents.append(dataset.cl_embeddings[text_i][-1].detach().cpu().numpy())
    first_latents = np.array(first_latents)
    last_latents = np.array(last_latents)
    return first_latents.mean(0), first_latents.std(0), last_latents.mean(0), last_latents.std(0)

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "--model_type",
    default=None,
    type=str,
    required=True,
    help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
)
parser.add_argument(
    "--model_name_or_path",
    default=None,
    type=str,
    required=True,
    help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
)

parser.add_argument("--prompt", type=str, default="")
parser.add_argument("--length", type=int, default=20)
parser.add_argument("--stop_token", type=str, default=None, help="Token at which text generation is stopped")

parser.add_argument(
    "--temperature",
    type=float,
    default=1.0,
    help="temperature of 1.0 has no effect, lower tend toward greedy sampling",
)
parser.add_argument(
    "--repetition_penalty", type=float, default=1.0, help="primarily useful for CTRL model; in that case, use 1.2"
)
parser.add_argument("--k", type=int, default=0)
parser.add_argument("--num-sentences", type=int, default=0)
parser.add_argument("--split-sentences", type=int, default=1)
parser.add_argument("--multiply-sentences", type=int, default=1)
parser.add_argument("--p", type=float, default=0.99)

parser.add_argument("--prefix", type=str, default="", help="Text added prior to input.")
parser.add_argument("--padding_text", type=str, default="", help="Deprecated, the use of `--prefix` is preferred.")

parser.add_argument("--no_eos", action="store_false", help="Avoid using CUDA when available")
parser.add_argument("--dryrun", action="store_true", default=False, help="Text added prior to input.")
parser.add_argument("--suppress_eos", action="store_true", default=False, help="Text added prior to input.")
parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")
parser.add_argument("--dataset_name", type=str, default="", help="Text added prior to input.")
parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
parser.add_argument("--fixed_prompt", action="store_true", help="Avoid using CUDA when available")
parser.add_argument("--num_return_sequences", type=int, default=1, help="The number of samples to generate.")
parser.add_argument("--num_intervals", type=int, default=1, help="The number of samples to generate.")
parser.add_argument("--block_size", type=int, default=1024)
parser.add_argument("--use_dataset", action="store_true", default=False, help="Text added prior to input.")
parser.add_argument("--project", type=str, default="", help="Text added prior to input.")
parser.add_argument("--encoder_filepath", type=str, required=True,default="", help="Text added prior to input.")
parser.add_argument("--latent_dim", type=int, default=3, help="random seed for initialization")
parser.add_argument("--use_random_embs", action="store_true", help="Avoid using CUDA when available")
parser.add_argument("--use_true_end_latent", action="store_true", help="Avoid using CUDA when available")
parser.add_argument("--label", type=str, default="", help="Text added prior to input.")

parser.add_argument("--method", type=str, default="", help="Text added prior to input.")
parser.add_argument("--first_sentence", action="store_true", help="Avoid using CUDA when available")
parser.add_argument("--full_section", action="store_true", help="Avoid using CUDA when available")
parser.add_argument("--autoregressive", action="store_true", help="Avoid using CUDA when available")
parser.add_argument(
    "--fp16",
    action="store_true",
    help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
)

_StoreTrueAction(option_strings=['--fp16'], dest='fp16', nargs=0, const=True, default=False, type=None, choices=None, help='Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit', metavar=None)

In [8]:
nseed = 2022
path2repo = '/home/sheng136/workspace/myprojects/language_modeling_via_stochastic_processes/'

domain = "tickettalk"
latent_dim = '16'


args = parser.parse_args(f"""--model_type=gpt2 
                         --model_name_or_path={path2repo}/language_modeling_via_stochastic_processes/transformers/examples/pytorch/language-modeling/LM_{domain}_{latent_dim}/ 
                         --prompt="<|endoftext|>" 
                         --num_return_sequences=1 
                         --num_intervals=1000 
                         --method=sample 
                         --stop_token="<|endoftext|>" 
                         --dataset_name={domain} 
                         --encoder_filepath={path2repo}/trained_model/{domain}_encoder.ckpt 
                         --latent_dim={latent_dim} 
                         --project=LM_{domain} 
                         --no_eos
                         --label=LM_{domain}_{latent_dim} 
                         --seed={nseed}""".split())

In [9]:
args.device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
args.use_section_null = 0

logger.warning(f"device: {args.device}, n_gpu: {args.n_gpu}, 16-bits training: {args.fp16}")

set_seed(args)

11/16/2022 19:43:40 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False


In [10]:
# Initialize the model and tokenizer
try:
    args.model_type = args.model_type.lower()
    model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
except KeyError:
    raise KeyError("the model {} you specified is not supported. You are welcome to add it and open a PR :)")

tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)
model = model_class.from_pretrained(args.model_name_or_path)
model.to(args.device)

model.transformer._config.use_contrastive_embeddings = True

if args.suppress_eos:
    bad_words_ids = [[tokenizer.eos_token_id]]
else:
    bad_words_ids = None

if args.no_eos:
    min_length = 1023
else:
    min_length= 10 # default value

In [11]:
SECTION_IDS, SPECIAL_TOKENS, tokenizer = get_special_tokens(
        dataset_name=args.dataset_name, tokenizer=tokenizer)

model.transformer.special_tokens = SPECIAL_TOKENS

Old tokenizer size:  50260
Not adding because it's already contained
New tokenizer size:  50260


## Encoder

In [12]:
base_model = 'gpt2'
CL_MODEL = get_checkpoint(
        dataset_name=args.dataset_name,
        latent_dim=args.latent_dim,
        sec_id=True,
        token_size=len(tokenizer),
        base_model=base_model,
        filepath=args.encoder_filepath
    )# .to(cpu_device)
CL_MODEL.to(args.device)
CL_MODEL.eval()

GPT2OUEncoder(
  (model): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        

## Data

In [13]:
assert args.dataset_name
from language_modeling_via_stochastic_processes.src import constants
print(f'Args: {args}')
train_path, _, eval_path = get_data_paths(args)
train_dataset = get_dataset(
    args=args,
    tokenizer=tokenizer,
    file_path=train_path,
    special_words=SECTION_IDS,
    cache_dir=constants.PATH2HUGGINGFACE,
    cl_model=CL_MODEL,
)

Args: Namespace(autoregressive=False, block_size=1024, dataset_name='tickettalk', device=device(type='cuda'), dryrun=False, encoder_filepath='/home/sheng136/workspace/myprojects/language_modeling_via_stochastic_processes//trained_model/tickettalk_encoder.ckpt', first_sentence=False, fixed_prompt=False, fp16=False, full_section=False, k=0, label='LM_tickettalk_16', latent_dim=16, length=20, method='sample', model_name_or_path='/home/sheng136/workspace/myprojects/language_modeling_via_stochastic_processes//language_modeling_via_stochastic_processes/transformers/examples/pytorch/language-modeling/LM_tickettalk_16/', model_type='gpt2', multiply_sentences=1, n_gpu=1, no_cuda=False, no_eos=False, num_intervals=1000, num_return_sequences=1, num_sentences=0, p=0.99, padding_text='', prefix='', project='LM_tickettalk', prompt='"<|endoftext|>"', repetition_penalty=1.0, seed=2022, split_sentences=1, stop_token='"<|endoftext|>"', suppress_eos=False, temperature=1.0, use_dataset=False, use_random_e

/home/sheng136/workspace/myprojects/language_modeling_via_stochastic_processes/language_modeling_via_stochastic_processes/transformers/src/transformers/data/datasets/language_modeling.py:64: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


num examples 2376
num filtered 0
Lengths
examples
<|endoftext|> [ USER ] hi....am buying a ticket tonight so we go and see a movie at AMC mountain 16 [ ASSISTANT ] No problem. Is there a particular type of movie you’re looking for? [ USER ] hhhmmmmm not at all. i dont have any in mind for now [ ASSISTANT ] Sure. I can help with that. Let me listings at AMC Mercado 24. [ USER ] sure you can but i want to see the movie at AMC mountain 16 [ ASSISTANT ] Oh, sorry about that. So you’re interested in action films at AMC Mountain 16, right? [ USER ] yeah [ ASSISTANT ] OK. I show one action movie playing at AMC Mountain 16: No Time To Die. Remaining showtimes are 4:30pm, 6:40pm and 9:10pm. Does any of those work? [ USER ] yeah but 9.10pm will be perfect for me [ ASSISTANT ] Great. And how many tickets? [ USER ] myself and two other persons are going to see a movie [ ASSISTANT ] All right. Let me confirm that you’d like three tickets for No Time To Die at AMC Mountain 16 tonight at 9:10pm. Is t

In [29]:
eval_dataset.special_words

['[ USER ]', '[ ASSISTANT ]', ' . ']

In [14]:
eval_dataset = get_dataset(
        args=args,
        tokenizer=tokenizer,
        file_path=eval_path,
        special_words=SECTION_IDS,
        cache_dir=constants.PATH2HUGGINGFACE,
        cl_model=CL_MODEL,
    )

LOADING MOVIE TM
num examples 1188
num filtered 0
Lengths
examples
<|endoftext|> [ USER ] hai sir, we all are like to watch a comedy movie.. we dont like action movie.. i think.. is there is there movie is action movie? [ ASSISTANT ] No problem. Is there a particular type of movie you’re looking for? [ USER ] wow supper... i dont expect this... are you sure sir this movie is a comedy movie? then if are u help me to listinings at amc mercada 24 [ ASSISTANT ] Sure. I can help with that. Let me listings at AMC Mercado 24. [ USER ] sir again i told you... i want watch comedy movie.. is there is the amc mountain 16 is right or wrong? [ ASSISTANT ] Oh, sorry about that. So you’re interested in action films at AMC Mountain 16, right? [ USER ] okay sir.. i was compromise my family to watch action movie now... because i want to see tha amc mountain 16 now [ ASSISTANT ] OK. I show one action movie playing at AMC Mountain 16: No Time To Die. Remaining showtimes are 4:30pm, 6:40pm and 9:10pm. Does

In [17]:
# Estimate density for last sentence
# Average density for all training samples
first_latent_mu, first_latent_std, last_latent_mu, last_latent_std = get_density(dataset=train_dataset, lm=model, cl_model=CL_MODEL)

2376


In [133]:
first_latent_mu

array([-0.12817074,  0.11731824,  0.08505533, -0.12896992,  0.13096477,
        0.12195772,  0.12528765, -0.10445517,  0.11606154,  0.13890566,
       -0.13555746,  0.15454775, -0.13034433,  0.12845673,  0.14243399,
        0.13648628], dtype=float32)

In [18]:
last_latent_std

array([0.20150964, 0.19876935, 0.1916553 , 0.1879684 , 0.19223699,
       0.19855234, 0.19538742, 0.19835728, 0.2010545 , 0.19634782,
       0.19453451, 0.19091173, 0.1909538 , 0.18812436, 0.19124323,
       0.19830342], dtype=float32)

In [20]:
num_intervals = len(eval_dataset)

In [21]:
num_intervals

1188

In [22]:
print("last latent mu", last_latent_mu)
print("last latent std", last_latent_std)

last latent mu [-0.52328944  0.5134945   0.5024361  -0.508544    0.5275813   0.5180528
  0.5092071  -0.51341796  0.5401892   0.52739847 -0.5112348   0.53281057
 -0.51549774  0.5339563   0.5201655   0.5258357 ]
last latent std [0.20150964 0.19876935 0.1916553  0.1879684  0.19223699 0.19855234
 0.19538742 0.19835728 0.2010545  0.19634782 0.19453451 0.19091173
 0.1909538  0.18812436 0.19124323 0.19830342]


In [23]:
print("Checking example embeddings: {}".format(eval_dataset.cl_embeddings[0][0]))
print("Checking example embeddings: {}".format(eval_dataset.cl_embeddings[0][-1]))
print("Checking example embeddings: {}".format(eval_dataset.cl_embeddings[-1][0]))
print("Checking example embeddings: {}".format(eval_dataset.cl_embeddings[-1][-1]))

Checking example embeddings: tensor([-0.1766,  0.1473,  0.1222, -0.1754,  0.1808,  0.1625,  0.1590, -0.1451,
         0.1540,  0.1787, -0.1800,  0.1995, -0.1605,  0.1790,  0.1835,  0.1909],
       device='cuda:0')
Checking example embeddings: tensor([-0.7280,  0.7133,  0.6992, -0.6989,  0.7188,  0.7198,  0.7100, -0.7148,
         0.7379,  0.7346, -0.7065,  0.7187, -0.7052,  0.7223,  0.7213,  0.7226],
       device='cuda:0')
Checking example embeddings: tensor([-0.1025,  0.0956,  0.0594, -0.1013,  0.0859,  0.0948,  0.1046, -0.0729,
         0.0904,  0.1108, -0.1112,  0.1333, -0.1023,  0.0926,  0.1122,  0.0967],
       device='cuda:0')
Checking example embeddings: tensor([-0.2416,  0.2438,  0.2398, -0.2420,  0.2593,  0.2482,  0.2493, -0.2351,
         0.2556,  0.2598, -0.2406,  0.2530, -0.2498,  0.2663,  0.2442,  0.2473],
       device='cuda:0')


In [31]:
eval_dataset.special_words

['[ USER ]', '[ ASSISTANT ]', ' . ']

In [24]:
len(eval_dataset.cl_embeddings[0])

408

In [25]:
len(train_dataset.cl_embeddings[0])

289

In [26]:
eval_dataset.cl_texts[1]

"[ USER ] I would like to buy three tickets for tonight. [ ASSISTANT ] No problem. Is there a particular type of movie you’re looking for? [ USER ] My two buds and I are going. Probably an action movie. Nothing mushy! [ ASSISTANT ] Sure. I can help with that. Let me check listings at AMC Mercado 24. [ USER ] Oh, Pardon me. We wanted to go to the AMC Mountain 16. [ ASSISTANT ] Oh, sorry about that. So you’re interested in action films at AMC Mountain 16, right? [ USER ] You are correct! [ ASSISTANT ] OK. I see one action movie playing at AMC Mountain 16: No Time To Die. Remaining showtimes are 4:30pm, 6:40pm and 9:10pm. Does any of those work? [ USER ] That sounds good. We'd probably want to go to the 9:10pm showing. [ ASSISTANT ] Great. And how many tickets? [ USER ] My friends Joe, Bob, and I are going to the movie. So, 3. [ ASSISTANT ] All right. Let me confirm that you’d like three tickets for No Time To Die at AMC Mountain 16 tonight at 9:10pm. Is that all correct? [ USER ] Yes, th

In [27]:
eval_dataset.raw_texts[1]

"<|endoftext|> [ USER ] I would like to buy three tickets for tonight. [ ASSISTANT ] No problem. Is there a particular type of movie you’re looking for? [ USER ] My two buds and I are going. Probably an action movie. Nothing mushy! [ ASSISTANT ] Sure. I can help with that. Let me check listings at AMC Mercado 24. [ USER ] Oh, Pardon me. We wanted to go to the AMC Mountain 16. [ ASSISTANT ] Oh, sorry about that. So you’re interested in action films at AMC Mountain 16, right? [ USER ] You are correct! [ ASSISTANT ] OK. I see one action movie playing at AMC Mountain 16: No Time To Die. Remaining showtimes are 4:30pm, 6:40pm and 9:10pm. Does any of those work? [ USER ] That sounds good. We'd probably want to go to the 9:10pm showing. [ ASSISTANT ] Great. And how many tickets? [ USER ] My friends Joe, Bob, and I are going to the movie. So, 3. [ ASSISTANT ] All right. Let me confirm that you’d like three tickets for No Time To Die at AMC Mountain 16 tonight at 9:10pm. Is that all correct? [ 

In [41]:
import random
num_example = random.sample(range(num_intervals), k = 1)[0]

In [42]:
num_example

80

In [43]:
# 1188
row = eval_dataset.cl_texts[num_example]
row = row.replace('<newline>', '')
row = row.replace(' , ', ', ')
row = row.strip() # NOTE: remove break line
row = ' '.join(row.split()) # remove multiple spaces
split_pattern = " . "
split_text = row.split(split_pattern)[:-1]
split_text = [ _ + split_pattern for _ in split_text ]

In [44]:
split_text

[]

In [46]:
true_cl_feats = torch.stack(eval_dataset.cl_embeddings[num_example])

In [49]:
true_cl_feats.shape

torch.Size([339, 16])

In [48]:
true_cl_feats = true_cl_feats[::args.split_sentences]

In [85]:
LABELS = ['TRUE CL', 'BRIDGE CL (DE)','RANDOM CL'
]
# INTERPOLATION - BRIDGE
print(f"DENSITY ESTIMATE: {last_latent_mu}")
print(f"DENSITY ESTIMATE STD: {last_latent_std}")
B_T = np.random.normal(loc=last_latent_mu, scale=last_latent_std)

DENSITY ESTIMATE: [-0.52328944  0.5134945   0.5024361  -0.508544    0.5275813   0.5180528
  0.5092071  -0.51341796  0.5401892   0.52739847 -0.5112348   0.53281057
 -0.51549774  0.5339563   0.5201655   0.5258357 ]
DENSITY ESTIMATE STD: [0.20150964 0.19876935 0.1916553  0.1879684  0.19223699 0.19855234
 0.19538742 0.19835728 0.2010545  0.19634782 0.19453451 0.19091173
 0.1909538  0.18812436 0.19124323 0.19830342]


In [51]:
len(true_cl_feats)

339

In [52]:
num_sentences = len(true_cl_feats) if not args.split_sentences else int(len(true_cl_feats)/float(args.split_sentences))
num_sentences *= args.multiply_sentences

In [53]:
num_sentences

339

In [56]:
actual_inputs = eval_dataset.examples[num_example]

In [57]:
end = eval_dataset.get_end_points(actual_inputs)

In [58]:
len(end)

23

In [59]:
num_sentences = 40

In [60]:
print("Original num sentences: {}".format(len(end)))
print("Target num sentences: {}".format(num_sentences))
print("min length", min_length)
end_lengths = [end[i] if i == 0 else end[i+1] - end[i] for i in range(len(end)-1)]
end_lengths = (np.array(end_lengths)*(num_sentences/len(end)))
#end_lengths = np.ones(end_lengths.shape)
end_lengths = end_lengths.astype(np.int)

Original num sentences: 23
Target num sentences: 40
min length 10


In [61]:
end_lengths

array([29, 19, 29, 19, 27,  5, 22, 26, 66, 19, 31, 17, 12, 15, 26, 62,  5,
       20,  5, 48, 10, 15])

In [226]:
for row in range(len(train_dataset.cl_embeddings[0])):
    X[row] = train_dataset.cl_embeddings[0][row].cpu().numpy()

In [62]:
end_lengths

array([29, 19, 29, 19, 27,  5, 22, 26, 66, 19, 31, 17, 12, 15, 26, 62,  5,
       20,  5, 48, 10, 15])

In [89]:
#bridge_feats = simulate_brownian_bridge(B_0=true_cl_feats[0], B_T=B_T, num_samples=num_sentences,sentence_lengths=end_lengths)
bridge_feats = simulate_brownian_bridge(
                B_0=true_cl_feats[0], B_T=B_T, num_samples=num_sentences,
                sentence_lengths=end_lengths
            )

bridge_feats = torch.tensor(bridge_feats, dtype=true_cl_feats.dtype).to(args.device)

# RANDOM
random_feats = torch.rand(true_cl_feats.shape).to(args.device)
feats = [true_cl_feats, bridge_feats, random_feats]

In [90]:
from generation_metrics import GenerationMetrics

In [91]:
fname = args.model_name_or_path.split('/')[-2]
args.encoder_type = 'contrast'

In [92]:
fname

'LM_tickettalk_16'

In [93]:
gt_cl_tracker = GenerationMetrics(model=model, device=args.device,
                                tokenizer=tokenizer, dataset_name=args.dataset_name,
                                fname=fname+"_trueCLEmbs_" + args.method,
                                model_args=args,
                                subclass="GT")
random_cl_tracker = GenerationMetrics(model=model, device=args.device,
                                tokenizer=tokenizer, dataset_name=args.dataset_name,
                            model_args=args,
                                fname=fname+"_randomCLEmbs_"+args.method,
                             subclass="RANDOM")
bridge_cl_tracker = GenerationMetrics(model=model, device=args.device,
                                tokenizer=tokenizer, dataset_name=args.dataset_name,
                                fname=fname+"_bridgeCLEmbs_"+args.method,
                            model_args=args,
                                subclass="BRIDGE")


In [94]:
prompt_text = args.prompt if args.prompt else ""

In [95]:
trackers = [gt_cl_tracker, bridge_cl_tracker,
            random_cl_tracker
                    ]

In [96]:
prompt_text

'"<|endoftext|>"'

In [97]:
for seq_i, (seq_cl_feats, tracker) in enumerate(zip(feats, trackers)):
    cl_feats = seq_cl_feats[0] # Get the first sentence feat
    prefix = args.prefix if args.prefix else args.padding_text
    encoded_prompt = tokenizer.encode(prefix + prompt_text, add_special_tokens=True, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(args.device)

    if encoded_prompt.size()[-1] == 0:
        input_ids = None
    else:
        input_ids = encoded_prompt

    if 'filter' in args.dataset_name:
        length = 1024
    else:
        length = 1024 # len(eval_dataset.examples[_])

            # RESET THE CL INDEX
    model.transformer._cur_cl_idx = 0
    model.transformer._has_reset = False

    max_length = min(length + len(encoded_prompt[0]), 1024)
    if args.no_eos:
        max_length = 1024

    if args.method == "sample":
        output_sequences = model.generate(
                    input_ids=input_ids,
                    section_ids=None,
                    cl_feats=cl_feats, # .to(args.device),
                    seq_cl_feats=seq_cl_feats,
                    max_length=max_length,
                    temperature=args.temperature,
                    top_k=args.k,
                    top_p=args.p,
                    repetition_penalty=args.repetition_penalty,
                    do_sample=True,
                    num_return_sequences=args.num_return_sequences,
                    bad_words_ids=bad_words_ids,
                    min_length=min_length-50
                )

    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []
    
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    # print(f"=== GENERATED SEQUENCE {generated_sequence_idx + 1} ===")
        original = torch.clone(generated_sequence)
        generated_sequence = generated_sequence.tolist()
        print("Generated length: {}".format(len(generated_sequence)))

        # Decode text
        # text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        text = tokenizer.decode(generated_sequence, skip_special_tokens=True)

        # Remove all text after the stop token
        text = text[: text.find(args.stop_token) if args.stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
        prompt_text + text[len(tokenizer.decode(encoded_prompt[0], skip_special_tokens=True)) :]
           )

        gt_raw_seq = eval_dataset.raw_texts[num_example]
        generated_sequences.append(total_sequence)
        print("[ GENERATED FOR {} ]: {}".format(LABELS[seq_i], total_sequence))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated length: 316
[ GENERATED FOR TRUE CL ]: "<|endoftext|>"Do you know what movie tickets are available for tonight for Sure Place in the Mountain AMC 16 Cinemark downtown?"  [ USER ]  Yes, sir. I would like to buy tickets to The Call of the Wild, which is playing at that theater.  [ ASSISTANT ]  Oh, wait, wait a second. I was referring to a new release, is this already done?  [ USER ]  Yes, then. Thank you.  [ ASSISTANT ]  Now, would you like me to fetch these tickets?  [ USER ]  Yes, please.  [ ASSISTANT ]  Okay. Correct?  [ USER ]  Sure.  [ ASSISTANT ]  To confirm, you would like to purchase one ticket for today's 6:20 PM showing of The Call of the Wild at the Mountain AMC 16 Cinemark downtown?  [ USER ]  Yes, please.  [ ASSISTANT ]  Okay? Hmmm. So about what time would you like to hear the movie?  [ USER ]  I would like to hear the show time at 12:00 PM for two.  [ ASSISTANT ]  Okay. How many tickets would you like to purchase?  [ USER ]  Three tickets. Just one ticket.  [ ASS

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated length: 205
[ GENERATED FOR BRIDGE CL (DE) ]: "<|endoftext|>"Esquire Petite  [ ASSISTANT ]  Hello!  [ USER ]  And can I get three tickets for what kind of movie is playing currently?  [ ASSISTANT ]  There are 3 movies playing currently. The first movie is Action which is rated PG 13. The second movie is Rotten Tomatoes which is rated R. And finally you have another movie called Horror which is rated R.  [ USER ]  Which one is Rotten Tomatoes?  [ ASSISTANT ]  Horror is rated R with a Rotten Tomatoes score of 61%  [ USER ]  Which one is Rotten Tomatoes, IRL, and which one is Rotten Tomatoes, I think?  [ ASSISTANT ]  This theater currently has 2 Action movies playing. The first is titled, "The Turning" and starts at 1pm, 3pm, 6pm and 8pm. The second is titled "The Turning" and starts at 1pm, 3pm, 6pm and 8pm.  [ USER ]  Thank you so much!  [ ASSISTANT ]  You're welcome. Go ahead and order them. 
Generated length: 359
[ GENERATED FOR RANDOM CL ]: "<|endoftext|>"Are there any thea

In [87]:
eval_dataset.raw_texts[num_example]

"<|endoftext|> [ USER ] hi i'd like to buy tickets tonight for the amc mountain 16 please. [ ASSISTANT ] OK. I understand that you’d like to get tickets for tonight. Can you tell me the city and state where the theater is located? [ USER ] it's california, santa barbara [ ASSISTANT ] OK. And are you interested in AMC Mountain 16 or AMC Mercado 24? [ USER ] i already said i wanted amc mountain 16. [ ASSISTANT ] Sorry for the trouble but I wanted to make sure I have it right. [ USER ] OK. [ ASSISTANT ] Is there a particular movie you’re looking for? [ USER ] can we skip to the end please? i'm in a hurry. [ ASSISTANT ] OK, so far I understand that you are interested in tickets for tonight at AMC Mountain 16. We just need to decide on a movie, the number of tickets and a showtime. [ USER ] what is playing at amc mountain 16 please. [ ASSISTANT ] I found two movies at AMC Mountain 16: No Time To Die and Mulan. [ USER ] tickets for no time to die please. [ USER ] i prefer 9:10pm [ ASSISTANT 